Assignment 3 - Question 1

For this task, you are given a dataset of flower images. The dataset contains
images of 9 types of flowers.

In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import numpy as np
import os

%matplotlib inline
from google.colab import drive
drive.mount('/content/drive')

train_images = np.load("./flower_subset.npz")["train_images"]
train_labels = np.load("./flower_subset.npz")["train_labels"]
test_images = np.load("./flower_subset.npz")["test_images"]
test_labels = np.load("./flower_subset.npz")["test_labels"]

#Train Images: (1556, 128, 128)
#Train Labels: (1556)
#Test Images: (90, 128, 128)
#Test Labels: (90)


train_images_resized = []
test_images_resized = []
#Resize from 128x128 to 64x64 (loses a lot of image quality)
for a in range(train_images.shape[0]):
  train_images_resized.append(cv2.resize(train_images[a], dsize=(64, 64), interpolation=cv2.INTER_AREA))
for b in range(test_images.shape[0]):
  test_images_resized.append(cv2.resize(test_images[b], dsize=(64, 64), interpolation=cv2.INTER_AREA))

#Train Images: (1556, 64, 64)
#Test Images: (90, 64, 64)
#Labels are the same

Mounted at /content/drive


In [14]:
#HOG Featurs Function
#Input: a (x,64,64) picures
#Output: a vector of size 1600*x
def HoGFeatures(images):  
  img_size = (64, 64) # h x w in pixels
  cell_size = (8, 8)  # h x w in pixels
  block_size = (4, 4)  # h x w in cells
  nbins = 4  # number of orientation bins

  # create HoG Object
  # winSize is the size of the image cropped to multiple of the cell size
  # all arguments should be given in terms of number of pixels
  #winsize=(64,64)
  #blocksize=(32,32)
  #blockstride=(8,8)
  #cellsize=(4,4)
  #nbins=4
  hog = cv2.HOGDescriptor(_winSize=(img_size[1] // cell_size[1] * cell_size[1],
                                  img_size[0] // cell_size[0] * cell_size[0]),
                        _blockSize=(block_size[1] * cell_size[1],
                                    block_size[0] * cell_size[0]),
                        _blockStride=(cell_size[1], cell_size[0]),
                        _cellSize=(cell_size[1], cell_size[0]),
                        _nbins=nbins)
  features = []
  for i in range(len(images)):
     features.append(hog.compute((images[i]*255).astype(np.uint8)).reshape(1, -1))
  features = np.vstack(features)
  #features, train: (1556,1600)
  #features, test: (90,1600)
  return features

In [18]:
def casAcc(x,y):
  wrong = 0
  for i in range(len(x)):
    if (x[i] != y[i]):
      wrong = wrong + 1
  return ((len(x) - wrong) / len(x)) * 100


In [19]:
from sklearn import svm
train_features = HoGFeatures(train_images_resized)
test_features = HoGFeatures(test_images_resized)
clf = svm.SVC(gamma='auto', C=1) #with c=1 get very bad results
clf.fit(train_features, train_labels)
# now predict the label test images
result = clf.predict(test_features)
print("Predicted Label:", result)
print("Actual Label:", test_labels)
print("Accuracy:", casAcc(result, test_labels))

Predicted Label: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Actual Label: [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3
 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 7
 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8]
Accuracy: 11.11111111111111


In [26]:
#increasing c seems to have diminishing returns as it increases, will hold it at about 80 for the next test
#got best results for G around 0.01
#got over 50% many times, ex. gamma is: 0.0085 , C is: 80
g = "auto" #normally 1/n_features
c = 1
for i in range(1,21):
  c=i*10;
  print("gamma is:",g,", C is:",c)
  clf = svm.SVC(gamma=g, C=c) 
  clf.fit(train_features, train_labels)
  # now predict the label test images
  result = clf.predict(test_features)
  print("Accuracy:", casAcc(result, test_labels))
for i in range(1,21):
  c=80;
  g=i*0.0005
  print("gamma is:",g,", C is:",c)
  clf = svm.SVC(gamma=g, C=c) 
  clf.fit(train_features, train_labels)
  # now predict the label test images
  result = clf.predict(test_features)
  print("Accuracy:", casAcc(result, test_labels))

gamma is: auto , C is: 10
Accuracy: 30.0
gamma is: auto , C is: 20
Accuracy: 42.22222222222222
gamma is: auto , C is: 30
Accuracy: 50.0
gamma is: auto , C is: 40
Accuracy: 48.888888888888886
gamma is: auto , C is: 50
Accuracy: 51.11111111111111
gamma is: auto , C is: 60
Accuracy: 54.44444444444444
gamma is: auto , C is: 70
Accuracy: 53.333333333333336
gamma is: auto , C is: 80
Accuracy: 55.55555555555556
gamma is: auto , C is: 90
Accuracy: 55.55555555555556
gamma is: auto , C is: 100
Accuracy: 55.55555555555556
gamma is: auto , C is: 110
Accuracy: 54.44444444444444
gamma is: auto , C is: 120
Accuracy: 53.333333333333336
gamma is: auto , C is: 130
Accuracy: 53.333333333333336
gamma is: auto , C is: 140
Accuracy: 53.333333333333336
gamma is: auto , C is: 150
Accuracy: 54.44444444444444
gamma is: auto , C is: 160
Accuracy: 54.44444444444444
gamma is: auto , C is: 170
Accuracy: 54.44444444444444
gamma is: auto , C is: 180
Accuracy: 55.55555555555556
gamma is: auto , C is: 190
Accuracy: 55.

In [31]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=10, max_depth=5, criterion='entropy')
clf.fit(train_features, train_labels)

result = clf.predict(test_features)
print("Predicted Label:", result)
print("Actual Label:", test_labels)
print("Accuracy:", casAcc(result, test_labels))

Predicted Label: [4 4 4 4 4 3 8 4 4 4 1 5 1 5 1 0 1 6 1 1 4 4 2 2 2 2 2 2 6 1 4 2 8 1 6 2 3
 1 2 6 4 7 1 4 4 4 7 4 4 4 4 4 4 5 8 2 5 7 4 4 3 1 2 7 2 6 6 1 2 6 1 1 0 1
 4 1 1 4 4 1 4 2 4 0 8 4 0 8 4 4]
Actual Label: [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3
 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 7
 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8]
Accuracy: 30.0


In [36]:
#Got best results with a depth of 10 and num estimators around 100-200: over 50% many times
#ex. num_estimators is: 130 , max depth is: 10 gave over 50
est = 10
depth = 2
for a in range(1,6):
  for b in range(1, 21):
    est = b * 10
    depth = a * 2
    print("num_estimators is:",est,", max depth is:",depth)
    clf = RandomForestClassifier(n_estimators=est, max_depth=depth, criterion='entropy') 
    clf.fit(train_features, train_labels)

    result = clf.predict(test_features)
    print("Accuracy:", casAcc(result, test_labels))


num_estimators is: 10 , max depth is: 2
Accuracy: 25.555555555555554
num_estimators is: 20 , max depth is: 2
Accuracy: 24.444444444444443
num_estimators is: 30 , max depth is: 2
Accuracy: 24.444444444444443
num_estimators is: 40 , max depth is: 2
Accuracy: 23.333333333333332
num_estimators is: 50 , max depth is: 2
Accuracy: 22.22222222222222
num_estimators is: 60 , max depth is: 2
Accuracy: 18.88888888888889
num_estimators is: 70 , max depth is: 2
Accuracy: 24.444444444444443
num_estimators is: 80 , max depth is: 2
Accuracy: 23.333333333333332
num_estimators is: 90 , max depth is: 2
Accuracy: 23.333333333333332
num_estimators is: 100 , max depth is: 2
Accuracy: 20.0
num_estimators is: 110 , max depth is: 2
Accuracy: 22.22222222222222
num_estimators is: 120 , max depth is: 2
Accuracy: 23.333333333333332
num_estimators is: 130 , max depth is: 2
Accuracy: 23.333333333333332
num_estimators is: 140 , max depth is: 2
Accuracy: 25.555555555555554
num_estimators is: 150 , max depth is: 2
Accur

In [43]:
#test with different random states
import random
random.seed(10)
for i in range(0,30):   
    r_state = random.randint(1,500)
    print("Random State Number is:", r_state)
    clf = svm.SVC(gamma='auto', C=80, random_state=r_state) 
    clf.fit(train_features, train_labels)
    result = clf.predict(test_features)
    print("Accuracy of svm:", casAcc(result, test_labels))
    clf = RandomForestClassifier(n_estimators=100, max_depth=10, criterion='entropy', random_state=r_state) 
    clf.fit(train_features, train_labels)
    result = clf.predict(test_features)
    print("Accuracy of random forest:", casAcc(result, test_labels))


Random State Number is: 293
Accuracy of svm: 55.55555555555556
Accuracy of random forest: 45.55555555555556
Random State Number is: 17
Accuracy of svm: 55.55555555555556
Accuracy of random forest: 45.55555555555556
Random State Number is: 220
Accuracy of svm: 55.55555555555556
Accuracy of random forest: 51.11111111111111
Random State Number is: 248
Accuracy of svm: 55.55555555555556
Accuracy of random forest: 53.333333333333336
Random State Number is: 296
Accuracy of svm: 55.55555555555556
Accuracy of random forest: 51.11111111111111
Random State Number is: 8
Accuracy of svm: 55.55555555555556
Accuracy of random forest: 48.888888888888886
Random State Number is: 106
Accuracy of svm: 55.55555555555556
Accuracy of random forest: 51.11111111111111
Random State Number is: 237
Accuracy of svm: 55.55555555555556
Accuracy of random forest: 53.333333333333336
Random State Number is: 417
Accuracy of svm: 55.55555555555556
Accuracy of random forest: 54.44444444444444
Random State Number is: 252


Compare results of SVM and RF classifiers. Which one provides better
results? Experiment training both classifiers with a range of random states
(try different random values for the argument ‘random state’) and measure
classification accuracy of the test set. Which classifier is more stable or
robust to the change in random state?

With the values I tested atleast SVM seemed to consistently give me better results. I only tested random states from 1-500 and considering it represents a split between train and test subsets among the training data values like 230 and 240 should give similar results. We see that SVM never changes as the random number changes, while random forst will change with two different enough random numbers very often. Therefore SVM seems to be much more robust to changes in random state.
